In [45]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  
site = 'cstec_org_cn_publication'
site_name = 'China Science and Technology Exchange Center (China)'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
skipped_due_to_html_content =0
unable_to_fetch_cat_url = 0
publish_source = 'cstec.org.cn'
country = 'China'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

headers = {
                'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
        }

#http://cstec.org.cn/recentnews.html?column=1027
response = requests.get('http://cstec.org.cn/handlers/cistcMenuInfoListceshi.ashx?columnid=1027&isall=1&keyword=&year=&pagenum=1').json()
for i in response['infolist']:
    # source_link
#     print(i)
    source_link = i['InfoUrl']
    if 'http' not in source_link:
        source_link  = 'http://cstec.org.cn/' + source_link 
    print(source_link)


   # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue 

    logger.info(f'Fetching {source_link}\n')
    time.sleep(random.randint(1,3))


    # source_headline
    source_headline = i["InfoTitle"]
    print(source_headline)

    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

    # Date and time
    pub_date, publish_time = '', current_time

    try:
        date_time_str = i["InfoPublicDate"] 
        date_time_obj = datetime.strptime(date_time_str, '%Y/%m/%d')
        ist_date_time = date_time_obj + timedelta(hours = 0,minutes = 0)  
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
     
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue
    # break if date is not today's date
#         if pub_date != created_on:
#             break

    # journalist   
    journalist = ''
    if not journalist: journalist = 'NA'        

    # source_content          
    source_content = i["InfoContent"]
    source_content = c.strip_html(source_content)

    # skip if content not found
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')
        skipped_due_to_content += 1
        continue

    # current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content =  i["InfoContent"]
    if not html_content:
        logger.info(f'Skipping due to html content {source_link}\n')
        skipped_due_to_html_content += 1
        continue 
    # image_urls
    image_urls = []

    # creating dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":foot_fall,
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

    cl_data.insert_one(clientdata)  
    no_of_data += 1   
    break
    
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country : {country}\n')
logger.info(f'Skipped due to html content: {skipped_due_to_html_content}\n')
logger.info(f'language : {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')


http://cstec.org.cn/infoDetail.html?id=96813&column=1027
科技创新交流与合作展示专区亮相第二十届中国国际人才交流大会网上虚拟展厅


{'MenuIsPublish': None,
 'MenuIsDeleted': None,
 'Infotheme': None,
 'InfoId': 96813,
 'InfoMenu': 0,
 'InfoTitle': '科技创新交流与合作展示专区亮相第二十届中国国际人才交流大会网上虚拟展厅',
 'InfoViceTitle': '',
 'InfoUrl': 'infoDetail.html?id=96813&column=1027',
 'InfoContent': '<p><span style="font-family: 宋体, SimSun;">&nbsp;&nbsp;2022年5月26日下午，第二十届中国国际人才交流大会（以下简称“大会”）在线开幕。本届大会由科技部（国家外国专家局）和深圳市人民政府共同主办，中国国际人才交流中心和中国科学技术交流中心等单位共同承办。</span></p><p><span style="font-family: 宋体, SimSun;">&nbsp;&nbsp;为聚焦科技创新人才交流与民间科技创新合作，大会设置“科技创新交流与合作展示专区”，由中国科学技术交流中心牵头组织。</span></p><p style="text-align: center;"><span style="font-family: 宋体, SimSun;"><img src="/ueditor/net/../../../CSTECUploadFiles/image/20220608/16546562513512128.png" title="&#555;&#65533;&#65533;1.png" alt="&#555;&#65533;&#65533;1.png" width="747" height="472" style="width: 747px; height: 472px;"/></span></p><p><span style="font-family: 宋体, SimSun;">&nbsp;&nbsp;专区设置“‘一带一路’创新合作”“与港澳台创新合作”“民间科技创新合作”“科技创新与产业合作”四个展区，重点展示“一带一路”联合实验室、国际杰青计划、“一带一路”科普交流、香港“创科博览”、澳门科技周，以及我国与美大、欧洲

In [48]:
k ='<img src="/ueditor/net/../../../CSTECUploadFiles/image/20220608/16546562513512128.png"'
# http://cstec.org.cn/CSTECUploadFiles/image/20220608/16546562513512128.png

## this is how to subsitute the specail character in python

In [56]:
re.sub('<img src=(.*)\.\.\/','<img src="http://cstec.org.cn/',k)

'<img src="http://cstec.org.cn/CSTECUploadFiles/image/20220608/16546562513512128.png"'

In [24]:
import requests
cat= 'http://cstec.org.cn/handlers/cistcMenuInfoList.ashx?columnid=1029&topN=4'
response=requests.get(cat).json()

In [27]:
for i in response['infolist']:
    print(i)
    

{'MenuIsPublish': 'True', 'MenuIsDeleted': 'False', 'Infotheme': None, 'InfoId': 96810, 'InfoMenu': 0, 'InfoTitle': '服务科研一线 促进国际合作--中国科学技术交流中心、科技日报社启动向科研人员赠报活动', 'InfoViceTitle': None, 'InfoUrl': 'infoDetail.html?id=96810&column=1029', 'InfoContent': None, 'InfoPublicDate': '2022/6/2', 'InfoProvider': None, 'InfoPerson': None, 'InfoPicture': None, 'InfoPictures': None}
{'MenuIsPublish': 'True', 'MenuIsDeleted': 'False', 'Infotheme': None, 'InfoId': 96786, 'InfoMenu': 0, 'InfoTitle': '科技部中国科学技术交流中心关于2022年度面向社会公开招聘面试安排的通知', 'InfoViceTitle': None, 'InfoUrl': 'infoDetail.html?id=96786&column=1029', 'InfoContent': None, 'InfoPublicDate': '2022/5/17', 'InfoProvider': None, 'InfoPerson': None, 'InfoPicture': None, 'InfoPictures': None}
{'MenuIsPublish': 'True', 'MenuIsDeleted': 'False', 'Infotheme': None, 'InfoId': 96780, 'InfoMenu': 0, 'InfoTitle': '科技部中国科学技术交流中心关于2022年度公开招聘高校毕业生面试安排的通知', 'InfoViceTitle': None, 'InfoUrl': 'infoDetail.html?id=96780&column=1029', 'InfoContent': None, 'InfoPubl

In [29]:
response['infolist']

[{'MenuIsPublish': 'True',
  'MenuIsDeleted': 'False',
  'Infotheme': None,
  'InfoId': 96810,
  'InfoMenu': 0,
  'InfoTitle': '服务科研一线 促进国际合作--中国科学技术交流中心、科技日报社启动向科研人员赠报活动',
  'InfoViceTitle': None,
  'InfoUrl': 'infoDetail.html?id=96810&column=1029',
  'InfoContent': None,
  'InfoPublicDate': '2022/6/2',
  'InfoProvider': None,
  'InfoPerson': None,
  'InfoPicture': None,
  'InfoPictures': None},
 {'MenuIsPublish': 'True',
  'MenuIsDeleted': 'False',
  'Infotheme': None,
  'InfoId': 96786,
  'InfoMenu': 0,
  'InfoTitle': '科技部中国科学技术交流中心关于2022年度面向社会公开招聘面试安排的通知',
  'InfoViceTitle': None,
  'InfoUrl': 'infoDetail.html?id=96786&column=1029',
  'InfoContent': None,
  'InfoPublicDate': '2022/5/17',
  'InfoProvider': None,
  'InfoPerson': None,
  'InfoPicture': None,
  'InfoPictures': None},
 {'MenuIsPublish': 'True',
  'MenuIsDeleted': 'False',
  'Infotheme': None,
  'InfoId': 96780,
  'InfoMenu': 0,
  'InfoTitle': '科技部中国科学技术交流中心关于2022年度公开招聘高校毕业生面试安排的通知',
  'InfoViceTitle': None,
  'InfoUrl':

In [20]:
response

{'infolist': [{'MenuIsPublish': None,
   'MenuIsDeleted': None,
   'Infotheme': None,
   'InfoId': 96813,
   'InfoMenu': 0,
   'InfoTitle': '科技创新交流与合作展示专区亮相第二十届中国国际人才交流大会网上虚拟展厅',
   'InfoViceTitle': '',
   'InfoUrl': 'infoDetail.html?id=96813&column=1027',
   'InfoContent': '<p><span style="font-family: 宋体, SimSun;">&nbsp;&nbsp;2022年5月26日下午，第二十届中国国际人才交流大会（以下简称“大会”）在线开幕。本届大会由科技部（国家外国专家局）和深圳市人民政府共同主办，中国国际人才交流中心和中国科学技术交流中心等单位共同承办。</span></p><p><span style="font-family: 宋体, SimSun;">&nbsp;&nbsp;为聚焦科技创新人才交流与民间科技创新合作，大会设置“科技创新交流与合作展示专区”，由中国科学技术交流中心牵头组织。</span></p><p style="text-align: center;"><span style="font-family: 宋体, SimSun;"><img src="/ueditor/net/../../../CSTECUploadFiles/image/20220608/16546562513512128.png" title="&#555;&#65533;&#65533;1.png" alt="&#555;&#65533;&#65533;1.png" width="747" height="472" style="width: 747px; height: 472px;"/></span></p><p><span style="font-family: 宋体, SimSun;">&nbsp;&nbsp;专区设置“‘一带一路’创新合作”“与港澳台创新合作”“民间科技创新合作”“科技创新与产业合作”四个展区，重点展示“一带一路”联合实验室、国际杰青计划、“一带一路”

In [40]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options



options = Options()
options.headless = True
browser = webdriver.Chrome(executable_path=r"C:\Users\KIT\Desktop\webcrawling\chromedriver.exe")
browser.maximize_window()
try:
    browser.get('http://cstec.org.cn/infoDetail.html?id=96813&column=1027')
except:
    print('not reachable')
page = browser.page_source

C:\Users\KIT\AppData\Local\Temp\ipykernel_12492\3535049925.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=r"C:\Users\KIT\Desktop\webcrawling\chromedriver.exe")


In [41]:
page

'<html><head lang="en">\n    <meta charset="UTF-8">\n    <title>中国科学技术交流中心</title>\n    <link rel="stylesheet" href="css/top-copyright.css">\n    <link rel="stylesheet" href="css/dtxx.css">\n    <link rel="stylesheet" href="css/bootstrap.css">\n    <link rel="stylesheet" href="CSS/index.css">\n    <script type="text/javascript" src="js/jquery-3.2.1.min.js"></script>\n    <script type="text/javascript" src="js/bootstrap.js"></script>\n    <script type="text/javascript" src="js/iframe.js"></script>\n    <script type="text/javascript" src="js/CistcMenuInfo.js"></script>\n    <script type="text/javascript" src="js/cistcInfoList.js"></script>\n    <script type="text/javascript" src="js/cistcNavbar.js"></script>\n    <script type="text/javascript" src="js/cistcInfoDetail.js"></script>\n    <script type="text/javascript" src="js/cistcInfoPicDetail.js"></script>\n    <script language="javascript">\n        // 显示信息详情\n        function ShowInfoDetailByID(infoid, insertPanelID, insertTitleID, con